In [251]:
#Imnporting libraries
import pandas as pd
import numpy as np
import os
from scipy.spatial.distance import cosine

#Listing the existing files in the dir
for root, dirs, files in os.walk("."):  
    for filename in files:
        print(filename)

1st_assgnmt_collab.ipynb
1st_assignment_intro.ipynb
2nd_assignment_collab.ipynb
2nd_assignment_intro.ipynb
README.md
COMMIT_EDITMSG
config
description
FETCH_HEAD
HEAD
index
ORIG_HEAD
applypatch-msg.sample
commit-msg.sample
fsmonitor-watchman.sample
post-update.sample
pre-applypatch.sample
pre-commit.sample
pre-push.sample
pre-rebase.sample
pre-receive.sample
prepare-commit-msg.sample
update.sample
exclude
HEAD
master
master
a6f461b1cda82bc2bc64e2f8e752c594cb4b79
5889fd52bdb928eb7b8ef26232706c784a7fb1
8a81d4f821e4a1d51a8c68599d74959a23bb7d
ff03ce761e5a44ccc2b2a825bf574c1bee0631
44966c920b4d862ad58a805a0ece2382922184
05b0aaa2d7ebb03a3e0fb3d246cda98a5b004a
96b16af7d370ec7985b75b69d3af36a913954d
2974eb78edd82b431c3690f7464a503c694356
2041e1411e121aea1a74646ec651e7c9c954db
807046e3874e227078d513b2e90156c2ab1414
fdc207848247f45b640d1b9e61f889fd73a974
9dd7992c843ab1040b900da7e3335ed0ff5593
4eb72f8826545861a52a4848f147bf97669990
2edfd2574cdc6b5ee4d4d59f8f97f67da4b8a1
668c30a6f4c99ca88cfa10f9b0

In [252]:
items = pd.read_csv('./files/ratings_2nd.csv')
items = items.set_index('User')
items_set = items.iloc[:20,:20]
calc_1 = items_set.multiply(items_set.iloc[:,0],axis=0).sum()
items_dist = items.iloc[20]
calc_2 = items_dist.multiply(items_dist.iloc[0], axis=0)
calc_3 = calc_1 / calc_2; calc_3.drop('1: Toy Story (1995)').sort_values(ascending=False)

260: Star Wars: Episode IV - A New Hope (1977)             0.747409
780: Independence Day (ID4) (1996)                         0.690665
296: Pulp Fiction (1994)                                   0.667846
318: Shawshank Redemption, The (1994)                      0.667424
1265: Groundhog Day (1993)                                 0.661016
1210: Star Wars: Episode VI - Return of the Jedi (1983)    0.644995
2916: Total Recall (1990)                                  0.623056
34: Babe (1995)                                            0.618070
3578: Gladiator (2000)                                     0.587852
356: Forrest Gump (1994)                                   0.580540
593: Silence of the Lambs, The (1991)                      0.570229
2028: Saving Private Ryan (1998)                           0.534579
2571: Matrix, The (1999)                                   0.505010
1259: Stand by Me (1986)                                   0.492659
1198: Raiders of the Lost Ark (1981)            

Task: Provide 5 movies most similar to Toy Story, in decreasing order of similarity using cosine similarity between items using raw (unnormalized) ratings. Only provide the movie ID, not the title.

Solution: 260, 780, 296, 318, and 1265 are the most similart to Toy Story.

In [362]:
filled = items_set.fillna(0)
#Initiating the empty df
movies_sim = pd.DataFrame(index=filled.columns, columns=filled.columns)
#Calculating cosine (another apprapproach to the above)
num_titles = len(movies_sim.columns)
for i in range(0, num_titles):
    for j in range(0, num_titles):
        movies_sim.iloc[i,j] = 1 - cosine(filled.iloc[:,i], filled.iloc[:,j])  

#Figuring out the algorithm on the example of one movie as we know the prediction for user 755
test = items_set.loc['755']
movies_sim.iloc[0]
nom = test.multiply(movies_sim.loc['780: Independence Day (ID4) (1996)']).sum()
mask = test.notnull()
denom = test.mask(mask, movies_sim.loc['780: Independence Day (ID4) (1996)']).sum()
nom/denom

#Doing the actual calc
target = items_set.loc['5277']
merged = pd.DataFrame(target).join(movies_sim)
nominator = merged.iloc[:,1:].multiply(merged.iloc[:,0], axis=0).sum()
#Replacing those movies that the user has not rated with NaN
lst = merged.iloc[:,0][merged.iloc[:,0].isnull()].reset_index().iloc[:,0].tolist()
for col in lst:
    merged.loc[:,[col]] = np.nan
denominator = merged.iloc[:,1:].sum(axis=1)
calc = nominator / denominator; calc.sort_values(ascending=False).head(5)

,5277
1: Toy Story (1995),1.0
1210: Star Wars: Episode VI - Return of the Jedi (1983),NaN
356: Forrest Gump (1994),NaN
"318: Shawshank Redemption, The (1994)",2.0
"593: Silence of the Lambs, The (1991)",4.0
3578: Gladiator (2000),2.0
260: Star Wars: Episode IV - A New Hope (1977),5.0
2028: Saving Private Ryan (1998),NaN
296: Pulp Fiction (1994),NaN
1259: Stand by Me (1986),4.0


Task: Provide the top 5 recommended movies for user 5277, using an average of the user’s ratings weighted by similarity to each candidate movie. You do not need to exclude movies they have rated. Consider all movies with nonnegative similarities (do not limit neighborhood size). Similarity between items is estimated using cosine similarity between items using raw (unnormalized) ratings.

Solution: The top five movies for user 5277 are 527, 1259, 260, 593 and 2396.

In [287]:
#Normalizing scores by the item average
user_mean = items_set.mean(axis=1)
items_mean = items_set.mean(axis=0)
norm_rating = items_set.subtract(user_mean, axis=0)
calc_1 = norm_rating.multiply(norm_rating.iloc[:,0],axis=0).sum()
eql_dist = np.sqrt(np.sum(np.square(norm_rating)))
calc_2 = eql_dist.multiply(eql_dist.iloc[0], axis=0)
calc_3 = calc_1 / calc_2; calc_3.drop('1: Toy Story (1995)').sort_values(ascending=False).head(5)


34: Babe (1995)                          0.554448
356: Forrest Gump (1994)                 0.355780
296: Pulp Fiction (1994)                 0.295013
318: Shawshank Redemption, The (1994)    0.215975
2028: Saving Private Ryan (1998)         0.192799
dtype: float64

Task: Provide 5 movies most similar to Toy Story, in decreasing order of similarity, by computing item similarities using cosine similarity using adjusted (normalized) ratings. Only provide the movie ID, not the title.

Solution: 34, 356, 296, 318, and 2028 are the most similart to Toy Story.

In [418]:
norm_rating_filled = items_set.subtract(user_mean, axis=0).fillna(0)
norm_rating = items_set.subtract(user_mean, axis=0)
#Initiating the empty df
movies_sim = pd.DataFrame(index=norm_rating_filled.columns, columns=norm_rating_filled.columns)
#Calculating cosine (another apprapproach to the above)
num_titles = len(movies_sim.columns)
for i in range(0, num_titles):
    for j in range(0, num_titles):
        movies_sim.iloc[i,j] = 1 - cosine(norm_rating_filled.iloc[:,i], norm_rating_filled.iloc[:,j]) 
movies_sim[movies_sim < 0] = 0        
target = norm_rating.loc['5277']
merged = pd.DataFrame(target).join(movies_sim)
nominator = merged.iloc[:,1:].multiply(merged.iloc[:,0], axis=0).sum()
#Replacing those movies that the user has not rated with NaN
lst = merged.iloc[:,0][merged.iloc[:,0].isnull()].reset_index().iloc[:,0].tolist()
for col in lst:
    merged.loc[:,[col]] = np.nan
denominator = merged.iloc[:,1:].sum(axis=1)
calc = (nominator / denominator); (calc +  2.769231).sort_values(ascending=False).head(5)

260: Star Wars: Episode IV - A New Hope (1977)             4.565822
527: Schindler's List (1993)                               4.562483
1259: Stand by Me (1986)                                   3.602142
2396: Shakespeare in Love (1998)                           3.297939
593: Silence of the Lambs, The (1991)                      3.220890
2028: Saving Private Ryan (1998)                           3.095918
356: Forrest Gump (1994)                                   2.904472
1198: Raiders of the Lost Ark (1981)                       2.861378
296: Pulp Fiction (1994)                                   2.804427
1210: Star Wars: Episode VI - Return of the Jedi (1983)    2.789618
2571: Matrix, The (1999)                                   2.681204
541: Blade Runner (1982)                                   2.641633
3578: Gladiator (2000)                                     2.324026
780: Independence Day (ID4) (1996)                         2.043164
2762: Sixth Sense, The (1999)                   

Task: Provide the top 5 recommended movies for user 5277, using an average of the user’s ratings weighted by similarity to each candidate movie. You do not need to exclude movies they have rated. Consider all movies with nonnegative similarities (do not limit neighborhood size). Similarity between items is estimated using cosine similarity between items using adjusted (normalized) ratings.

Solution: Movies 260, 527, 1259 and 2396 are the top predicting movies for user 5277.